In [1]:
import fiftyone as fo
fo.list_datasets()

['open-images-v7-train-validation-test-1000']

In [1]:
%pip install screeninfo

  Using cached screeninfo-0.8.1-py3-none-any.whl.metadata (2.9 kB)
Using cached screeninfo-0.8.1-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from screeninfo import get_monitors

def get_screen_resolution():
    monitor_info = get_monitors()[1]
    return monitor_info.width, monitor_info.height

width, height = get_screen_resolution()
print(f'Screen resolution is {width}x{height}')

Screen resolution is 1920x1080


In [1]:
from skimage.metrics import structural_similarity as ssim
import cv2
from time import time

def compare_images(imageA, imageB, width=512, height=512):
    # Конвертировать изображения в grayscale
    imageA = cv2.resize(imageA, (width, height))
    imageB = cv2.resize(imageB, (width, height))
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    # Вычислить SSIM между двумя изображениями
    ssim_start = time()
    result = ssim(imageA, imageB)
    print(f'SSIM took {time() - ssim_start} seconds')
    return result

# Загрузить изображения
imageA = cv2.imread("examples/1_1.png")
imageB = cv2.imread("examples/1_2.png")
# imageA = cv2.imread("examples/2_1.png")
# imageB = cv2.imread("examples/2_2.png")
# imageA = cv2.imread("examples/3_1.png")
# imageB = cv2.imread("examples/3_2.png")

# Сравнить изображения
similarity = compare_images(imageA, imageB)

print("The similarity between the two images is: ", similarity)

SSIM took 0.02542424201965332 seconds
The similarity between the two images is:  0.19225333504105774


In [2]:
import os
import cv2
import shutil
from skimage.metrics import structural_similarity as ssim
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict

def compare_images(imageA, imageB):
    if imageA is None or imageB is None:
        return 0
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    return ssim(imageA, imageB)

image_dir = "x:/frames"
move_dir = "x:/frames/similar"
threshold = 0.9


In [ ]:

image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]
images = {f: cv2.resize(cv2.imread(os.path.join(image_dir, f)), (312, 312)) for f in tqdm_notebook(image_files, desc="Loading images")}
print(f"Loaded {len(images)} images")


In [40]:
to_move = set()
for i, (nameA, imageA) in  enumerate(tqdm_notebook(images.items(), desc="Comparing images overall")):
    for nameB, imageB in tqdm_notebook(list(images.items())[i+1:], desc="Comparing images"):
        if nameB not in to_move and compare_images(imageA, imageB) > threshold:
            to_move.add(nameB)
            
print(f"Found {len(to_move)} similar images")

os.makedirs(move_dir, exist_ok=True)
for name in to_move:
    shutil.move(os.path.join(image_dir, name), os.path.join(move_dir, name))

Comparing images overall:   0%|          | 0/4209 [00:00<?, ?it/s]

Comparing images:   0%|          | 0/4208 [00:00<?, ?it/s]

Comparing images:   0%|          | 0/4207 [00:00<?, ?it/s]

Comparing images:   0%|          | 0/4206 [00:00<?, ?it/s]

Comparing images:   0%|          | 0/4205 [00:00<?, ?it/s]

Comparing images:   0%|          | 0/4204 [00:00<?, ?it/s]

KeyboardInterrupt: 

Loading images...
Loaded 4209 images from 33 sources
Comparing images...


Comparing images from source VideoReader-10:  26%|██▌       | 31/119 [02:50<08:04,  5.50s/it]


KeyboardInterrupt: 

final

In [1]:
import os
import cv2
import shutil
from skimage.metrics import structural_similarity as ssim
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict
from multiprocessing import Pool

def compare_images(imageA, imageB):
    if imageA is None or imageB is None:
        return 0
    return ssim(imageA, imageB)

image_dir = "x:/frames"
move_dir = "x:/frames/similar"
threshold = 0.9

In [2]:
print("Loading images...")
image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]

# Group files by source name
source_files = defaultdict(list)
source_images = {}
for f in tqdm_notebook(image_files):
    source_name = f.split('_')[1]
    source_files[source_name].append(f)
    image = cv2.imread(os.path.join(image_dir, f))
    if image is None:
        print(f"Failed to load {f}")
        # удалить файл, который не удалось загрузить .png и .txt
        # вернуть имя файла без расширения
        file_name = os.path.splitext(f)[0]
        # удалить файл .png
        os.remove(os.path.join(image_dir, file_name + '.png'))
        # удалить файл .txt
        os.remove(os.path.join(image_dir, file_name + '.txt'))
        
        continue
    image = cv2.resize(image, (320, 320))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    source_images[f] = image

print(f"Loaded {len(image_files)} images from {len(source_files)} sources")

Loading images...


  0%|          | 0/4201 [00:00<?, ?it/s]

Loaded 4201 images from 33 sources


In [3]:
def process_source(args):
    source_name, files = args
    to_move = set()
    to_compare = list(files)  # Copy the list of files
    for i, nameA in enumerate(tqdm_notebook(to_compare, desc=f"Comparing images from source {source_name}")):
        imageA = source_images[nameA]
        print(f"Comparing with {nameA}")
        for nameB in tqdm_notebook(to_compare[i+1:], desc=f"Comparing with {nameA}", leave=False):
            imageB = source_images[nameB]
            if compare_images(imageA, imageB) > threshold:
                print(f"Found similar images: {nameA} and {nameB}")
                to_move.add(nameB)
                to_compare.remove(nameB)  # Remove the image from the list of images to compare
    print(f"Found {len(to_move)} similar images in source {source_name}")
    for name in to_move:
        shutil.move(os.path.join(image_dir, name), os.path.join(move_dir, name))
        txt_name = os.path.splitext(name)[0] + '.txt'
        txt_path = os.path.join(image_dir, txt_name)
        if os.path.exists(txt_path):
            shutil.move(txt_path, os.path.join(move_dir, txt_name))

In [4]:
with Pool(16) as p:
    p.map(process_source, source_files.items())

In [4]:
# Скрипт для конвертирования разметки в формат YOLO
from pathlib import Path
import shutil
from tqdm import tqdm

src_dir = Path('x:/test')
old_dir = Path('x:/test/old')

old_dir.mkdir(parents=True, exist_ok=True)

for src_file in tqdm(src_dir.glob('*.txt'), desc='Converting'):
    with src_file.open('r') as file:  # Указываем кодировку
        lines = file.readlines()
        
    if len(lines[0].strip().split()) > 5: # Если разметка ещё не в формате YOLO
        # Создаём копию файла со старой разметкой в папке old
        shutil.copy(src_file, old_dir)
        # Создаём новый файл с разметкой в формате YOLO
        with src_file.open('w') as file:
            for line in lines:
                x1, y1, x2, y2, id, conf = map(float, line.strip().split())
                x = (x1 + x2) / 2
                y = (y1 + y2) / 2
                w = x2 - x1
                h = y2 - y1
                file.write(f'{int(id)} {x} {y} {w} {h}\n')
    else:
        print(f'File {src_file} is already in YOLO format')

Converting: 17it [00:00, 208.31it/s]


In [2]:
import requests
from bs4 import BeautifulSoup
import random

def get_random_image_link(query):
    url = f"https://duckduckgo.com/html/?q={query.replace(' ', '+')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    image_elements = soup.find_all("img", class_="tile--img__img  js-lazyload")
    image_links = [img.get('data-src') for img in image_elements if img.get('data-src') is not None]
    return random.choice(image_links)

if __name__ == "__main__":
    queries = ["truck", "van", "truck CCTV", "trucks and vans CCVT"]
    query = random.choice(queries)
    image_link = get_random_image_link(query)
    print(image_link)

In [ ]:
import os
from urllib import parse
from urllib.request import urlretrieve
import requests as req
import bs4

search_words = ['truck', 'van']

img_dir = './img/'

for word in search_words:
    dir_path = img_dir + word
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

urlKeyword = parse.quote(word)

#Create url with target word
url = 'https://www.google.com/search?hl=jp&q=' + urlKeyword + '&btnG=Google+Search&tbs=0&safe=off&tbm=isch'

# headers is necessary when you send request
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0',}

#Request is made with url and headers
request = req.Request(url=url, headers=headers)
page = req.urlopen(request)

html = page.read().decode('utf-8')

#Use BeutifulSoup, “html.parser”. HTML object is available in Python.
html = bs4.BeautifulSoup(html, 'html.parser')

elems = html.select('.rg_meta.notranslate')

counter = 0
error_counter = 0

for ele in elems:
    ele = ele.contents[0].replace('"','').split(',')
eledict = dict()

#imageURL is indicated here.
for e in ele:
    num = e.find(':')
    eledict[e[0:num]] = e[num+1:]
    imageURL = eledict['ou']

    #URL retrieve: extract imageURL from file_path

    #Try catching image, if error occurs, execute except program
    try:
        file_path = dir_path + '/' + str(counter)+'.jpg'
        urlretrieve(imageURL, file_path)
        counter += 1
    except Exception as e:
        error_counter += 1
        if counter == 1:
            print('Start downloading', word)
        if counter==200:

# print('{} errors occur out of '.format(counter, error_counter))